In [1]:
import numpy as np
import pandas as pd

## Plotly plotting support
import plotly.plotly as py

# import plotly.offline as py
# py.init_notebook_mode()

import plotly.graph_objs as go
import plotly.figure_factory as ff

# Make the notebook deterministic 
np.random.seed(42)

# Feature Engineering

In this notebook we will explore a key part of data science: _**feature engineering** the process of transforming the representation of model inputs to enable better model approximation._  Feature engineering enables you to:

1. **encode** non-numeric features to be used as inputs to common numeric models
1. capture **domain knowledge** (e.g., the perceived loudness or sound is the log of the intensity)
1. **transform** complex relationships into simple linear relationships

---
<br/><br/><br/><br/><br/><br/>


## Mapping from Domain to Range

In the supervised learning setting were are given $(X,Y)$ paris with the goal of learning the mapping from $X$ to $Y$. For example, given pairs of square footage and price we want to learn a function that captures (or at least approximates) the relationship between square feet and price.  Our functional approximation is some form of typically parametric mapping from some **domain** to some **range**:

<img src="domain_range.png" width="400px">

In this class we will focus on **Multiple Regression** in which we consider mappings from potentially high-dimensional input spaces onto the real line (i.e., $y \in \mathbb{R}$):

<img src="domain_real_range.png" width="400px">

It is worth noting that this is distinct from **Multivariate Regression** in which we are predicting multiple (confusing?) response values (e.g., $y \in \mathbb{R}^q$).



# What is the Domain (Features)

Suppose we are given the following table:

<img src="input_table.png" width="600px">

Our goal is to learn a function that approximates the relationship between the blue and red columns.  Let's assume the range, `"Ratings"`, are the real numbers (this may be a problem if ratings are between [0, 5] but more on that later).

**What is the _domain_ of this function?**

---

<br/><br/><br/><br/><br/><br/>




The schema of the relational model provides one possible answer:

```sql
RatingsData(uid AS INTEGER, age AS FLOAT, 
            state AS STRING, hasBought AS BOOLEAN,
            review AS STRING, rating AS FLOAT)
```

Which would suggest that the domain is then:

$$
\textbf{Domain} = \mathbb{Z} \times \mathbb{R} \times \mathbb{S} \times \mathbb{B} \times \mathbb{S} \times \mathbb{R}
$$

Unfortunately, the techniques we have discussed so far and most of the techniques in machine learning and statistics operate on real-valued vector inputs $x \in \mathbb{R}^d$ (or for the statisticians $x \in \mathbb{R}^p$). 

### Goal: 

<img src="real_domain_range.png" width="400px">




Moreover, many of these techniques, especially the linear models we have been studying, assume the inputs are **continuous** variables in which the relative magnitude of the feature encode information about the response variable. 

In the following we define several basic transformations to encode features as real numbers.



---
<br/><br/><br/><br/><br/>


# Basic Feature Engineering:  _Get $\mathbb{R}$_

Our first step as feature engineers is to translate our data into a form that encodes each feature as a continuous variable.

## The _Uninformative_  Feature: `uid`

The `uid` was likely used to join the user information (e.g., `age`, and `state`) with some `Reviews` table.  The `uid` presents several questions:
* What is the meaning of the `uid` *number*? 
* Does the magnitude of the `uid` reveal information about the rating? 

There are several answers:

1. Although numbers, identifiers are **typically categorical** (like strings) and as a consequence the magnitude has little meaning.  In these settings we would either **drop** or **one-hot encode** the `uid`.  We will return to feature dropping and one-hot-encoding in a moment.

1. There are scenarios where the magnitude of the numerical `uid` value contains important information. When user ids are created in consecutive order, larger user ids would imply more recent users.  In these cases we might to interpret the `uid` feature as a real number. 



---
<br/><br/><br/><br/><br/>


## Dropping Features

While uncommon there are certain scenarios where manually dropping features might be helpful:

1. when the features **does not to contain information** associated with the prediction task.  Dropping uninformative features can help to address over-fitting, an issue we will discuss in great detail soon.  

1. when the feature is **not available when at prediction time.**  For example, the feature might contain information collected after the user entered a rating.  This is a common scenario in time-series analysis.

However in the absence of substantial domain knowledge, we would prefer to use algorithmic techniques to help eliminate features.  We will discuss this more when we return to regularization.


---
<br/><br/><br/><br/><br/>


## The _Continuous_ `age` Feature

The `age` feature encodes the users age.  This is already a continuous real number so no additional feature transformations are required.  However, as we will soon see, we may introduce additional related features (e.g., indicators for various age groups or non-linear transformations).

---
<br/><br/><br/><br/><br/>


## The _Categorical_ `state` Feature


The `state` feature is a string encoding the category (one of the 50 states).  How do we meaningfully encode such a feature as one or more real-numbers?

We could enumerate the states in alphabetical order `AL=0`, `AK=2`, ... `WY=49`.  This is a form of **dictionary encoding** which maps each category to an integer.  However, this would likely be a poor feature encoding since the magnitude provides little information about the rating.  

Alternatively, we might enumerate the states based on their geographic region (e.g., lower numbers for coastal states.). While this alternative dictionary encoding may provide information there is better way to encode categorical features for machine learning algorithms.

---
<br/><br/><br/><br/><br/><br/>

# One-Hot Encoding

<img src="one_hot_encoding.png" width="400px">



One-Hot encoding, sometimes also called **dummy encoding** is a simple mechanism to encode categorical data as real numbers such that the magnitude of each dimension is meaningful.  Suppose a feature can take on $k$ distinct values (e.g., $k=50$ for 50 states in the United Stated).  For each distinct _possible_ value a new feature (dimension) is created.  For each record, all the new features are set to zero except the one corresponding to the value in the original feature. 

The following is a relatively inefficient (why?) implementation:

In [2]:
def one_hot_encoding(x, categories):
    dictionary = dict(zip(categories, range(len(categories))))
    enc = np.zeros(len(categories))
    enc[dictionary[x]] = 1.0
    return enc

categories = ["cat", "dog", "apple"]
one_hot_encoding("dog", categories)

array([ 0.,  1.,  0.])

---
<br/><br/><br/><br/><br/><br/>


## One-Hot Encoding in Pandas

Here we create a toy dataframe of pets including their name and kind:

In [3]:
df = pd.DataFrame({
    "name": ["Goldy", "Scooby", "Brian", "Francine", "Goldy"],
    "kind": ["Fish", "Dog", "Dog", "Cat", "Dog"],
    "age": [0.5, 7., 3., 10., 1.]
}, columns = ["name", "kind", "age"])
df

,name,kind,age
0,Goldy,Fish,0.5
1,Scooby,Dog,7.0
2,Brian,Dog,3.0
3,Francine,Cat,10.0
4,Goldy,Dog,1.0


Pandas has a built in function to construct one-hot encodings called **`get_dummies`**

In [4]:
pd.get_dummies(df['kind'])

,Cat,Dog,Fish
0,0,0,1
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [5]:
pd.get_dummies(df)

,age,name_Brian,name_Francine,name_Goldy,name_Scooby,kind_Cat,kind_Dog,kind_Fish
0,0.5,0,0,1,0,0,0,1
1,7.0,0,0,0,1,0,1,0
2,3.0,1,0,0,0,0,1,0
3,10.0,0,1,0,0,1,0,0
4,1.0,0,0,1,0,0,1,0


**Issue:** While the Pandas `pandas.get_dummies` function is very convenient and even retains meaningful column labels it has one key downside.

The `get_dummies` function does not take the dictionary of possible values and so will not produce the same encoding if applied across multiple dataframes with different values. This can be a big issue when rendering predictions on a new dataset.

---

<br/><br/><br/><br/><br/>


## One-Hot Encoding in Scikit-Learn

Scikit-learn is a widely used machine learning package in Python and provides several implementations of feature encoders for categorical data. 

### DictVectorizer

The `DictVectorizer` encodes dictionaries by taking keys that map to strings and applying a one-hot encoding.

In [6]:
from sklearn.feature_extraction import DictVectorizer

vec_enc = DictVectorizer()
vec_enc.fit(df.to_dict(orient='records'))

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)

In [7]:
vec_enc.transform(df.to_dict(orient='records')).toarray()

array([[  0.5,   0. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ],
       [  7. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ],
       [  3. ,   0. ,   1. ,   0. ,   1. ,   0. ,   0. ,   0. ],
       [ 10. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,   0. ],
       [  1. ,   0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ]])

In [8]:
vec_enc.get_feature_names()

['age',
 'kind=Cat',
 'kind=Dog',
 'kind=Fish',
 'name=Brian',
 'name=Francine',
 'name=Goldy',
 'name=Scooby']

We can apply the dictionary vectorizer to new data:

In [9]:
vec_enc.transform([
    {"kind": "Cat", "name": "Goldy", "age": 35},
    {"kind": "Bird", "name": "Fluffy"},
    {"breed": "Chihuahua", "name": "Goldy"},
]).toarray()

array([[ 35.,   1.,   0.,   0.,   0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.]])

Notice that the second record `{"kind": "Bird", "name": "Fluffy"}` has invalid categories and missing fields and it's encoding is entirely zero.  Is this reasonable?

---

<br/><br/><br/><br/><br/><br/>

### _Bonus:_ sklearn `OneHotEncoder`

The basic sklearn `OneHotEncoder` encodes a column of integers corresponding to category values.  Therefore, we first need to **dictionary encode** the string values.  

In [10]:
# Convert the "kind" column into a category column
kind_codes = (
    df['kind'].astype("category", categories=["Cat", "Dog","Fish"])
        .cat.codes # Extract the category codes
)
kind_codes

0    2
1    1
2    1
3    0
4    1
dtype: int8

In [11]:
from sklearn.preprocessing import OneHotEncoder

# Build an instance of the encoder
onehot = OneHotEncoder()

# Construct an integer column vector from the 'kind_codes' column
column_vec_kinds = np.array([kind_codes.values]).T

# Fit the encoder (which can be resued to transform other data)
onehot.fit(column_vec_kinds)

# Transform the column vector
onehot.transform(column_vec_kinds).toarray()

array([[ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

---
<br/><br/><br/><br/><br/>

## Key Points on One-Hot Encoding

While one-hot encoding is the standard mechanism for encoding **categorical** data there are a few issues to keep in mind:

1. may generate **too many** dimensions/features
    1. sparse representations are often necessary
    1. watch out for issues with over-fitting (more on this soon)

1. all possible **values must be known in advance**
    1. unable introduce new categories when making predictions
    1. be sure to use the same encoding when making predictions

1. **missing values** are reasonably captured by a zero in all dummy features.

---

<br/><br/><br/><br/><br/>

## The _Boolean_ `hasBought` Feature

The `hasBought` feature is a boolean (0/1) valued feature but we it can have missing values:

<img src="input_table.png" width="600px">

There are a few options for encoding `hasBought`:

1. **Interpret directly as numbers.** If there were no missing values then the booleans are typically treated directly as continuous values.

1. **Apply one-hot encoding.** This would create two new features `hasBought=True` and `hasBought=False`.  This is probably the most general encoding but suffers from increased complexity.

1. **1/-1 Encoding.** Another common encoding for booleans with missing values is:

\begin{align}
\textbf{True} & \Rightarrow 1 \\
\textbf{Null} & \Rightarrow 0 \\
\textbf{False} & \Rightarrow -1 
\end{align}

---

<br/><br/><br/><br/><br/><br/>


## The _Text_ `review` Feature

Encoding text as a real-valued feature is especially challenging and many of the standard transformations are **lossy**. Moreover, all of the earlier transformations (e.g., one-hot encoding and Boolean representations) preserve the information in the feature. In contrast, most of the techniques for encoding text destroy information about the word order and in many cases key parts of the grammar.  

Here we will discuss two widely used representations of text:

* **Bag-of-Words Encoding**: encodes text by the frequency of each word
* **N-Gram Encoding**: encodes text by the frequency of sequences of words of length $N$

Both of these encoding strategies are related to the one-hot encoding with dummy features created for every word or sequence of words and with multiple dummy features having counts greater than zero.

---
<br/><br/><br/><br/><br/>



## The Bag-of-Words Encoding


The bag-of-words encoding is widely used and a standard representation for text in many of the popular text clustering algorithms.  The following is a simple illustration of the bag-of-words encoding:

<img src="bag_of_words.png" width="600px">

**Notice**
1. **Stop words are removed.** Stop-words are words like `is` and `about` that in isolation contain very little information about the meaning of the sentence.  Here is a good list of [stop-words in many languages](https://code.google.com/archive/p/stop-words/). 
1. **Word order information is lost.**  Nonetheless the vector still suggests that the sentence is about `fun`, `machines`, and `learning`.  Thought there are many possible meanings _learning machines have fun learning_ or _learning about machines is fun learning_ ...
1. **Capitalization and punctuation are typically removed.**  
1. **Sparse Encoding:** is necessary to represent the bag-of-words efficiently.  There are millions of possible words (including terminology, names, and misspellings) and so instantiating a `0` for every word that is not in each record would be incredibly inefficient.  

**Why is it called a bag-of-words?**  A bag is another term for a **multiset**: _an unordered 
collection which may contain multiple instances of each element._  


---
<br/><br/><br/><br/><br/><br/>

# Break?

When professor Gonzalez was a graduate student at Carnegie Mellon University, he and several other computer scientists created the following art piece on display at the Gates Center:

<img src="bag_of_words_art.jpg" width="300px">

**Notice**
1. The unordered collection of words in the bag.
1. The stop words on the floor.
1. _The missing broom._  The original sculpture had a broom attached but the janitor got confused .... 

---
<br/><br/><br/><br/><br/>

## The N-Gram Encoding

The N-Gram encoding is a generalization of the bag-of-words encoding designed to capture limited ordering information.  Consider the following passage of text:

> _The book was not well written but I did enjoy it._

If we re-arrange the words we can also write:

> _The book was well written but I did not enjoy it._

Moreover, local word order can be important when making decisions about text.  The n-gram encoding captures local word order by defining counts over sliding windows. In the following example a bi-gram ($n=2$) encoding is constructed:

<img src="ngram.png" width="800px">

The above n-gram would be encoded in the sparse vector:

<img src="ngram_vector.png" width="300px">

Notice that the n-gram captures key pieces of sentiment information: `"well written"` and `"not enjoy"`.  

N-grams are often used for other types of sequence data beyond text. For example, n-grams can be used to encode genomic data, protein sequences, and click logs. 

**N-Gram Issues**
1. The n-gram representation is hyper sparse and maintaining the dictionary of possible n-grams can be very costly.  The **hashing trick** is a popular solution to approximate the sparse n-gram encoding.  In the hashing trick each n-gram is mapped to a relatively large (e.g., 32bit) hash-id and the counts are associated with the hash index without saving the n-gram text in a dictionary.  As a consequence, multiple n-grams are treated as the same.
1. As $N$ increase the chance of seeing the same n-grams at prediction time decreases rapidly.


## Implementing Bag-of-words and N-grams 

In [12]:
frost_text = [x for x in """
Some say the world will end in fire,
Some say in ice.
From what Ive tasted of desire
I hold with those who favor fire.
""".split("\n") if len(x) > 0]

frost_text

['Some say the world will end in fire,',
 'Some say in ice.',
 'From what Ive tasted of desire',
 'I hold with those who favor fire.']

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Construct the tokenizer with English stop words
bow = CountVectorizer(stop_words="english")

# fit the model to the passage
bow.fit(frost_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
# Print the words that are kept
print("\nWords:", 
      list(zip(range(0,len(bow.get_feature_names())),bow.get_feature_names())))


Words: [(0, 'desire'), (1, 'end'), (2, 'favor'), (3, 'hold'), (4, 'ice'), (5, 'ive'), (6, 'say'), (7, 'tasted'), (8, 'world')]


In [15]:
print("\nSentence Encoding: \n")
# Print the encoding of each line
for (s, r) in zip(frost_text, bow.transform(frost_text)):
    print(s)
    print(r)
    print("------------------")


Sentence Encoding: 

Some say the world will end in fire,
  (0, 1)	1
  (0, 6)	1
  (0, 8)	1
------------------
Some say in ice.
  (0, 4)	1
  (0, 6)	1
------------------
From what Ive tasted of desire
  (0, 0)	1
  (0, 5)	1
  (0, 7)	1
------------------
I hold with those who favor fire.
  (0, 2)	1
  (0, 3)	1
------------------


In [16]:
# Construct the tokenizer with English stop words
bigram = CountVectorizer(stop_words="english", ngram_range=(1, 2))
# fit the model to the passage
bigram.fit(frost_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
# Print the words that are kept
print("\nWords:", 
      list(zip(range(0,len(bigram.get_feature_names())), bigram.get_feature_names())))


Words: [(0, 'desire'), (1, 'end'), (2, 'favor'), (3, 'hold'), (4, 'hold favor'), (5, 'ice'), (6, 'ive'), (7, 'ive tasted'), (8, 'say'), (9, 'say ice'), (10, 'say world'), (11, 'tasted'), (12, 'tasted desire'), (13, 'world'), (14, 'world end')]


In [18]:
print("\nSentence Encoding: \n")
# Print the encoding of each line
for (s, r) in zip(frost_text, bigram.transform(frost_text)):
    print(s)
    print(r)
    print("------------------")


Sentence Encoding: 

Some say the world will end in fire,
  (0, 1)	1
  (0, 8)	1
  (0, 10)	1
  (0, 13)	1
  (0, 14)	1
------------------
Some say in ice.
  (0, 5)	1
  (0, 8)	1
  (0, 9)	1
------------------
From what Ive tasted of desire
  (0, 0)	1
  (0, 6)	1
  (0, 7)	1
  (0, 11)	1
  (0, 12)	1
------------------
I hold with those who favor fire.
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
------------------


## _Bonus:_ Term Frequency Scaling

If we are encoding text in a particular domain (e.g., processing insurance claims) it is likely that there will be frequent terms (e.g., `insurance` or `claim`) that provide little information. However, because these terms occur frequently they can present challenges to some modeling techniques.  In these cases, additional scaling may be applied to transform the bag-of-word or n-gram vectors to emphasize the more informative terms. One of the most common scalings techniques is the **term frequency inverse document frequency (TF-IDF)** which emphasizes words that are unique to a particular record.  Because the notation is confusing, I have provided a pseudo code implementation.  However, you should use a more efficient sparse implementation like those provided in scikit learn.

```python
def tfidf(X):
    """
    Input: X is a bag of words matrix (rows=records, cols=terms)
    """
    (ndocs, nwords) = X.shape
    tf = X / X.sum(axis=1)[:, np.newaxis]
    idf = ndocs / (X > 0).sum(axis=0) 
    return tf * np.log(idf)
```


While these transformations are especially important when computing similarities between vector encodings of text.  We will not cover these transformations in DS100 but it is worth knowing that they exist.


# Summary of Feature Encoding

Most machine learning (ML) and statistics techniques operate on multivariate real-valued domains (i.e., vectors).  As a consequence, we need methods to encode non-continuous datatypes into meaningful continuous forms.  We discussed:

1. **one-hot** (a.k.a. **dummy variable**) encoding transform categorical values into vectors of binary values with dimension equal to the number of possible values.
1. **bag-of-words** and **n-gram** encoding transform text into frequency statistics for individual terms and groups of terms. 

We will now explore how feature transformations can be used to capture domain knowledge and encode complex relationships.

<br/><br/><br/><br/>




---

# Feature Transformations

In addition to transforming categorical and text features to real valued representations, we can also often improve model performance through the use of additional feature transformations.  Let's start with a simple toy example


## Linear Models for Non-Linear Data

To illustrate the potential for feature transformations consider the following *synthetic dataset*:

In [19]:
train_data = pd.read_csv("toy_training_data.csv")
print(train_data.describe())
train_data.head()

               X          Y
count  75.000000  75.000000
mean   -0.607319  -0.455212
std     6.120983  12.873863
min    -9.889558 -25.028709
25%    -6.191627 -10.113630
50%    -1.196950  -1.648253
75%     5.042387  10.910793
max     9.737739  22.921518


,X,Y
0,-9.889558,-7.221915
1,-9.588310,-10.111930
2,-9.312230,-15.816534
3,-9.095454,-19.059384
4,-9.070992,-22.349544


**Goal:** As usual we would like to learn a model that predicts $Y$ given $X$.

What does this relationship between $X \rightarrow Y$ look like?

In [20]:
train_points = go.Scatter(name = "Training Data", 
                          x = train_data['X'], y = train_data['Y'], 
                          mode = 'markers')
# layout = go.Layout(autosize=False, width=800, height=600)
py.iplot(go.Figure(data=[train_points]), 
         filename="L19_b_p1")

### Properties of the Data

How would you describe this data?

1. Is there a linear relationship between $X$ and $Y$?  
1. Are there other patterns?
1. How noisy is the data?
1. Do we see obvious outliers?

---

<br/><br/><br/><br/><br/>

# Least Squares Linear Regression  (Review)

For the remainder of this lecture we will focus on fitting least squares linear regression models.  Recall that **linear regression** models are functions of the form:

\begin{align}\large
f_\theta(x) = x^T \theta = \sum_{j=1}^p \theta_j x_j
\end{align}

and **least squares** implies a loss function of the form:

\begin{align} \large
L_\mathcal{D}(\theta) = \frac{1}{n} \sum_{i=1}^n \left(y_i - f_\theta(x) \right)^2
\end{align}

In the previous lecture, we derived the **normal equations** which define the loss minimizing $\hat{\theta}$ for: 

\begin{align}\large
\hat{\theta} 
& \large = \arg\min L_\mathcal{D}(\theta) \\
& \large = \arg\min \frac{1}{n} \sum_{i=1}^n \left(y_i - X \theta \right)^2 \\
& \large = \left( X^T X \right)^{-1} X^T Y
\end{align}


---

<br/><br/><br/><br/><br/>

## Linear Regression with scikit-learn

In this lecture we will use the [scikit-learn `linear_model` package](http://scikit-learn.org/stable/modules/linear_model.html) to compute the normal equations. This package supports a wide range of **generalized linear models**.  For those who are interested in studying machine learning, I would encourage you to skim through the descriptions of the various models in the `linear_model` package. These are the foundation of most practical applications of supervised machine learning.

In [21]:
from sklearn import linear_model

The following block of code creates an instance of the Least Squares Linear Regression model and the fits that model to the training data.  

In [22]:
line_reg = linear_model.LinearRegression(fit_intercept=True)
# Fit the model to the data
line_reg.fit(train_data[['X']], train_data['Y'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**Notice:** In the above code block we explicitly added a **bias** (**intercept**) term by setting `fit_intercept=True`.  Therefore we will not need to add an additional `constant` feature.

---

<br/><br/><br/><br/><br/>

### Plotting the Model

To plot the model we will predict the value of $Y$ for a range of $X$ values.  I will call these **query points** `X_query`.

In [23]:
X_query = np.linspace(-10, 10, 500)

Use the regression model to render predictions at each `X_query` point.

In [24]:
# Note that np.linspace returns a 1d vector therefore 
# we must transform it into a 2d column vector
line_Yhat_query = line_reg.predict(
    np.reshape(X_query, (len(X_query),1)))

To plot the residual we will also predict the $Y$ value for all the training points:

In [25]:
line_Yhat = line_reg.predict(train_data[['X']])

The following visualization code constructs a line as well as the residual plot

In [26]:
# Define the least squares regression line 
basic_line = go.Scatter(name = r"$\theta x$", x=X_query.T, 
                        y=line_Yhat_query)

# Definethe residual lines segments, a separate line for each 
# training point
residual_lines = [
    go.Scatter(x=[x,x], y=[y,yhat],
               mode='lines', showlegend=False, 
               line=dict(color='black', width = 0.5))
    for (x, y, yhat) in zip(train_data['X'], train_data['Y'], line_Yhat)
]

# Combine the plot elements
py.iplot([train_points, basic_line] + residual_lines, filename="L19_b_p2")

## Assessing the Fit

1. Is our simple linear model a good fit?  
1. What do we see in the above plot?
1. Are there clear outliers?  
1. Are there patterns to the residuals?

---

<br/><br/><br/><br/><br/>

# Residual Analysis

To help answer these questions it can often be helpful to plot the residuals in a **residual plot**.  Residual plots plot the difference between the predicted value and the observed value in response to a particular covariate ($X$ dimension).  The residual plot can help reveal patterns in the residual that might support additional modeling.


In [27]:
residuals = line_Yhat - train_data['Y'] 

In [28]:
# Plot.ly plotting code
py.iplot(go.Figure(
    data = [dict(x=train_data['X'], y=residuals, mode='markers')],
    layout = dict(title="Residual Plot", xaxis=dict(title="X"), 
                  yaxis=dict(title="Residual"))
), filename="L19_b_p3")

** Do we see a pattern? **

---
<br/><br/><br/><br/><br/>



## Using a Smoothing Model 

To better visualize the pattern we could apply another regression package. In the following plotting code we call a more sophisticated regression package `sklearn.kernel_ridge` to estimate a smoothed approximation to the residuals. 

In [29]:
from sklearn.kernel_ridge import KernelRidge

# Use Kernelized Ridge Regression with Radial Basis Functions to 
# compute a smoothed estimator.  Later in this notebook we will 
# actually implement part of this  ...
clf = KernelRidge(kernel='rbf', alpha=2)
clf.fit(train_data[['X']], residuals)
residual_smoothed = clf.predict(np.reshape(X_query, (len(X_query),1)))

# Plot the residuals with with a kernel smoothing curve
py.iplot(go.Figure(
    data = [dict(name = "Residuals", x=train_data['X'], y=residuals, 
                 mode='markers'),
            dict(name = "Smoothed Approximation", 
                 x=X_query, y=residual_smoothed, 
                 line=dict(dash="dash"))],
    layout = dict(title="Residual Plot", xaxis=dict(title="X"), 
                  yaxis=dict(title="Residual"))
), filename="L19_b_p4")

Again, the above plot suggests a cyclic pattern to the residuals. In higher dimensional settings or when many features are binary indicators (e.g., one-hot encoding) it will become difficult to interpret **residual plots**.  In these, cases we may instead examine the distribution of the residual to identify skew and outliers.

In [30]:
py.iplot(ff.create_distplot([residuals], group_labels=['Residuals']), 
         filename="L19_b_p5")

From the above plot we see that there are several large outliers and there  appears to be a gap just above 0.  

---

<br/><br/><br/><br/><br/>

## Cyclic Non-linearity

So we have what appears to be a non-linear cyclic structure.

In [31]:
py.iplot(go.Figure(
    data = [
        dict(name = "Residuals", x=train_data['X'], y=residuals, 
             mode='markers'),
        dict(name = "Smoothed Approximation", 
             x=X_query, y=residual_smoothed, 
             line=dict(dash="dash"))],
    layout = dict(title="Residual Plot", xaxis=dict(title="X"), 
                  yaxis=dict(title="Residual"))
), filename="L19_b_p4")

**Question:** Can we fit this non-linear cyclic structure with a linear model?

<br/><br/><br/>

---

## What does it mean to be a _linear model_


Let's return to what it means to be a linear model:

$$\large
f_\theta(x) = x^T \theta = \sum_{j=1}^p x_j \theta_j
$$

In what sense is the above model **linear**? 
1. Linear in the features $x$? 
1. Linear in the parameters $\theta$?

---
<br/><br/><br/><br/><br/> <br/>

The answer is **yes** to all the above questions!  

<br/><br/>

# Feature Functions

Consider the following alternative model formulation:

$$\large
f_\theta\left( \phi(x) \right) = \phi(x)^T \theta = \sum_{j=1}^{k} \phi(x)_j \theta_j
$$

where $\phi_j$ is an *arbitrary function* from $x\in \mathbb{R}^p$ to $\phi(x)_j \in \mathbb{R}$ and we define $k$ of these functions.  We often refer to these functions $\phi_j$ as **feature functions** or **basis functions** and their design plays a critical role in both how we capture prior knowledge and our ability to fit complicated data.

---

<br/><br/><br/><br/><br/>


# Capturing Domain Knowledge


Feature functions can be used to capture domain knowledge by:
1. Introducing additional information from other sources
1. Combining related features
1. Encoding non-linear patterns

Suppose I had data about customer purchases and I wanted to estimate their income:

\begin{align}
\phi(\text{date}, \text{lat}, \text{lon}, \text{amount})_1 
&= \textbf{isWinter}(\text{date}) \\
\phi(\text{date}, \text{lat}, \text{lon}, \text{amount})_2 
&= \cos\left( \frac{\textbf{Hour}(\text{date})}{12} \pi \right)  \\
\phi(\text{date}, \text{lat}, \text{lon}, \text{amount})_3 
&= \frac{\text{amount}}{\textbf{avg_spend}[\textbf{ZipCode}[\text{lat}, \text{lon}]]} \\
\phi(\text{date}, \text{lat}, \text{lon}, \text{amount})_4 
&= \exp\left(-\textbf{Distance}\left((\text{lat},\text{lon}),  \textbf{StoreA}\right)\right)^2 \\
\phi(\text{date}, \text{lat}, \text{lon}, \text{amount})_5 
&= \exp\left(-\textbf{Distance}\left((\text{lat},\text{lon}),  \textbf{StoreB}\right)\right)^2 
\end{align}

**Notice:** In the above feature functions:
1. The transformations are non-linear
1. They pull in additional information
1. They may encode implicit knowledge
1. The functions $\phi$ **do not** depend on $\theta$

---
<br/><br/><br/><br/><br/>

## Linear in $\theta$
As a consequence, while the model $f_\theta\left( \phi(x) \right)$ is no longer linear in $x$ it is still a **linear model** because it _is linear in $\theta$_.  This means we can continue to use the normal equations to compute the optimal parameters.  

To apply the normal equations we define the transformed feature matrix:

<img src="phi_matrix.png" width="400px">

Then substituting $\Phi$ for $X$ we obtain the **normal equation**:

$$ \large
\hat{\theta} = \left( \Phi^T \Phi \right)^{-1} \Phi^T Y
$$

It is worth noting that the model is also linear in $\Phi$ and that the $\phi_j$ form a new **basis** (hence the term **basis functions**) in which the data live.  As a consequence we can think of $\phi$ as mapping the data into a new (often higher dimensional space) in which the relationship between $y$ and $\phi(x)$ is defined by a **hyperplane**. 

---
<br/><br/><br/>

## Transforming the data with $\phi$

In our toy data set we observed a cyclic pattern.  Here we construct a $\phi$ to capture the cyclic nature of our data and visualize the corresponding hyperplane.


In the following cell we define a function $\phi$ that maps $x\in \mathbb{R}$ to the vector $[x,\sin(x)] \in \mathbb{R}^2$ 

$$ \large
\phi(x) = [x, \sin(x)]
$$

**Why not:**

$$ \large
\phi(x) = [x, \sin(\theta_3 x + \theta_4)]
$$

This would no longer be linear $\theta$.  However, in practice we might want to consider a range of $\sin$ basis:

$$ \large
\phi_{\alpha,\beta}(x) = \sin(\alpha x + \beta)
$$

for different values of $\alpha$ and $\beta$.  The parameters $\alpha$ and $\beta$ are typically called **hyperparameters** because (at least in this setting) they are not set automatically through learning.


In [32]:
def sin_phi(x):
    return [x, np.sin(x)]

We then compute the matrix $\Phi$ by applying $\phi$ to reach row (record) in the matrix $X$.  

In [33]:
Phi = np.array([sin_phi(x) for x in train_data['X']])

# Look at a few examples
Phi[:5,]

array([[-9.88955766,  0.44822588],
       [-9.58831011,  0.16280424],
       [-9.31222958, -0.11231092],
       [-9.09545422, -0.32340318],
       [-9.07099175, -0.34645201]])

It is worth noting that in practice we might prefer a more efficient "vectorized" version of the above code:
```python
Phi = np.vstack((train_data['X'], np.sin(train_data['X']))).T
```
however in this notebook we will use more explicit `for` loop notation.

---

<br/><br/><br/><br/><br/>

# Fit a _Linear Model_ on $\Phi$

We can again use the scikit-learn package to fit a linear model on the transformed space.

In [34]:
sin_reg = linear_model.LinearRegression(fit_intercept=False)
sin_reg.fit(Phi, train_data['Y'])

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [35]:
# Making predictions at the transformed query points
Phi_query = np.array([sin_phi(x) for x in X_query])
sin_Yhat_query = sin_reg.predict(Phi_query)

# plot the regression line
sin_line = go.Scatter(name = r"$\theta_0 x + \theta_1 \sin(x)$ ", 
                       x=X_query, y=sin_Yhat_query)

# Make predictions at the training points
sin_Yhat = sin_reg.predict(Phi)

# Plot the residual segments
residual_lines = [
    go.Scatter(x=[x,x], y=[y,yhat],
               mode='lines', showlegend=False, 
               line=dict(color='black', width = 0.5))
    for (x, y, yhat) in zip(train_data['X'], train_data['Y'], sin_Yhat)
]

py.iplot([train_points, sin_line, basic_line] + residual_lines, 
         filename="L19_b_p10")

Examine the residuals again

In [36]:
sin_Yhat = sin_reg.predict(Phi)
residuals = train_data['Y'] - sin_Yhat


# Use Kernelized Ridge Regression with Radial Basis Functions to 
# compute a smoothed estimator. 
clf = KernelRidge(kernel='rbf')
clf.fit(train_data[['X']], residuals)
residual_smoothed = clf.predict(np.reshape(X_query, (len(X_query),1)))

# Plot the residuals with with a kernel smoothing curve
py.iplot(go.Figure(
    data = [dict(name = "Residuals", 
                 x=train_data['X'], y=residuals, mode='markers'),
            dict(name = "Smoothed Approximation", 
                 x=X_query, y=residual_smoothed, 
                 line=dict(dash="dash"))],
    layout = dict(title="Residual Plot", 
                  xaxis=dict(title="X"), yaxis=dict(title="Residual"))
), filename="L19_b_p11")

Look at the distribution of residuals

In [37]:
py.iplot(ff.create_distplot([residuals], group_labels=['Residuals']), 
         filename="L19_b_p12")

**Recall** earlier that our residuals were more spread from -10 to 10 and now they have become more concentrated.  However, the outliers remain.  Is that a problem?

---
<br/><br/><br/><br/><br/><br/>

# A Linear Model in Transformed Space

As discussed earlier the model we just constructed, while non-linear in $x$ is actually a linear model in $\phi(x)$ and we can visualize that linear model's structure in higher dimensions.

In [38]:
# Plot the data in higher dimensions
phi3d = go.Scatter3d(name = "Raw Data",
    x = Phi[:,0], y = Phi[:,1], z = train_data['Y'],
    mode = 'markers',
    marker = dict(size=3),
    showlegend=False
)

# Compute the predictin plane
(u,v) = np.meshgrid(np.linspace(-10,10,5), np.linspace(-1,1,5))
coords = np.vstack((u.flatten(),v.flatten())).T
ycoords = coords @ sin_reg.coef_
fit_plane = go.Surface(name = "Fitting Hyperplane",
    x = np.reshape(coords[:,0], (5,5)),
    y = np.reshape(coords[:,1], (5,5)),
    z = np.reshape(ycoords, (5,5)),
    opacity = 0.8, cauto = False, showscale = False,
    colorscale = [[0, 'rgb(255,0,0)'], [1, 'rgb(255,0,0)']]
)

# Construct residual lines
Yhat = sin_reg.predict(Phi)
residual_lines = [
    go.Scatter3d(x=[x[0],x[0]], y=[x[1],x[1]], z=[y, yhat],
                 mode='lines', showlegend=False, 
                 line=dict(color='black'))
    for (x, y, yhat) in zip(Phi, train_data['Y'], Yhat)
]

    
# Label the axis and orient the camera
layout = go.Layout(
    scene=go.Scene(
        xaxis=go.XAxis(title='X'),
        yaxis=go.YAxis(title='sin(X)'),
        zaxis=go.ZAxis(title='Y'),
        aspectratio=dict(x=1.,y=1., z=1.), 
        camera=dict(eye=dict(x=-1, y=-1, z=0))
    )
)

py.iplot(go.Figure(data=[phi3d, fit_plane] + residual_lines, layout=layout), filename="L19_b_p14")

---
<br/><br/><br/><br/><br/>

# Loss Minimization 

Recall that in each stage of the process we have been minimizing the squared prediction error while increasing the sophistication of our model by introducing new features. When plotting the prediction error it is common to compute the **root mean squared error (RMSE)** which is the square-root of the average squared loss over the training data. 

$$ \large
\textbf{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(Y_i - f_\theta(X_i)\right)^2}
$$



In [39]:
def sq_loss(y, yhat):
    return np.mean((yhat-y)**2)
    
const_rmse = np.sqrt(sq_loss(train_data['Y'], train_data['Y'].mean()))
line_rmse = np.sqrt(sq_loss(train_data['Y'], line_Yhat))
sin_rmse = np.sqrt(sq_loss(train_data['Y'], sin_Yhat))

In [40]:
py.iplot(go.Figure(data =[go.Bar(
            x=[r'$\theta $', r'$\theta x$', 
               r'$\theta_0 x + \theta_1 \sin(x)$'],
            y=[const_rmse, line_rmse, sin_rmse]
            )], layout = go.Layout(title="Loss Comparison", 
                           yaxis=dict(title="RMSE"))), 
         filename="L19_b_p15")

By adding the `sine` feature function were were able to reduce the prediction error. How could we improve further?

# Generic Feature Functions

We will now explore a range of generic feature transformations. However before we proceed it is worth contrasting two categories of feature functions and their applications.

1. **Domain Specific Features:** In settings where our goal is to understand the model (e.g., identify important features that predict customer churn) we may want to construct meaningful features based on our understanding of the domain.  

1. **Generic Features:** However, in other settings where our primary goals is to make accurate predictions we may instead introduce generic feature functions that enable our models to fit _and generalize_ complex relationships. 

----

<br/><br/><br/><br/><br/><br/>


## The Polynomial Basis:

The first set of generic feature functions we will consider is the polynomial basis:

\begin{align}
\phi(x) = [x, x^2, x^3, \ldots, x^k]
\end{align}

We can define a generic python function to implement this basis:

In [41]:
def poly_phi(k):
    return lambda x: [x ** i for i in range(1, k+1)]

To simply the comparison of feature functions we define the following routine:

In [42]:
def evaluate_basis(phi, desc):
    # Apply transformation
    Phi = np.array([phi(x) for x in train_data['X']])
    
    # Fit a model
    reg_model = linear_model.LinearRegression(fit_intercept=False)
    reg_model.fit(Phi, train_data['Y'])
    
    # Create plot line
    X_test = np.linspace(-10, 10, 1000) # Fine grained test X
    Phi_test = np.array([phi(x) for x in X_test])
    Yhat_test = reg_model.predict(Phi_test)
    line = go.Scatter(name = desc, x=X_test, y=Yhat_test)
    
    # Compute RMSE
    Yhat = reg_model.predict(Phi)
    rmse = np.sqrt(sq_loss(train_data['Y'], Yhat))
    
    # return results
    return (line, rmse, reg_model)

### Starting with $k=5$ degree polynomials

In [43]:
(poly_line, poly_rmse, poly_reg) = (
    evaluate_basis(poly_phi(5), "Polynomial")
)

py.iplot(go.Figure(data=[train_points, poly_line, sin_line, basic_line], 
                   layout = go.Layout(xaxis=dict(range=[-10,10]), 
                                      yaxis=dict(range=[-25,25]))), 
         filename="L19_b_p16")

### Increasing to $k=15$

In [44]:
(poly_line, poly_rmse, poly_reg) = (
    evaluate_basis(poly_phi(15), "Polynomial")
)

py.iplot(go.Figure(data=[train_points, poly_line, sin_line, basic_line], 
                   layout = go.Layout(xaxis=dict(range=[-10,10]), 
                                      yaxis=dict(range=[-25,25]))), 
         filename="L19_b_p17")

Seems like a pretty reasonable fit.  Returning to the RMSE on the training data:

In [45]:
py.iplot([go.Bar(
            x=[r'$\theta $', r'$\theta x$', 
               r'$\theta_0 x + \theta_1 \sin(x)$', 
               'Polynomial'],
            y=[const_rmse, line_rmse, sin_rmse, poly_rmse]
    )], filename="L19_b_p18")

This was a slight improvement.  Perhaps we should increase to a higher degree polynomial? Why or why not?  We will return to this soon.

---
<br/><br/><br/><br/><br/>

## Gaussian Radial Basis Functions

One of the more widely used generic feature functions are Gaussian radial basis functions.  These feature functions take the form:

$$
\phi_{(\lambda, u_1, \ldots, u_k)}(x) = \left[\exp\left( - \frac{\left|\left|x-u_1\right|\right|_2^2}{\lambda} \right), \ldots, 
\exp\left( - \frac{\left|\left| x-u_k \right|\right|_2^2}{\lambda} \right) \right]
$$

The **hyper-parameters** $u_1$ through $u_k$ and $\lambda$ are not optimized with $\theta$ but instead are set externally.  In many cases the $u_i$ may correspond to points in the training data.  The term $\lambda$ defines the spread of the basis function and determines the "smoothness" of the function $f_\theta(\phi(x))$.

The following is a plot of three radial basis function centered at 2 with different values of $\lambda$.

In [46]:
def gaussian_rbf(u, lam=1):
    return lambda x: np.exp(-(x - u)**2 / lam)

In [47]:
tmpX = np.linspace(-2, 6,100)
py.iplot([
    dict(name=r"$\lambda=0.5$", x=tmpX, 
         y=gaussian_rbf(2, lam=0.5)(tmpX)),
    dict(name=r"$\lambda=1$", x=tmpX, 
         y=gaussian_rbf(2, lam=1.)(tmpX)),
    dict(name=r"$\lambda=2$", x=tmpX, 
         y=gaussian_rbf(2, lam=2.)(tmpX))
], filename="L19_b_p19")

### 10 RBF Functions $\lambda=1$

Here we plot 10 uniformly spaced RBF functions with $\lambda=1$

In [48]:
def rbf_phi(x):
    return [gaussian_rbf(u, 1.)(x) for u in np.linspace(-9, 9, 10)]

(rbf_line, rbf_rmse, rbf_reg) = evaluate_basis(rbf_phi, r"RBF")

py.iplot([train_points, rbf_line, poly_line, sin_line, basic_line], filename="L19_b_p20")

### 10 RBF Functions $\lambda=10$

In [49]:
def rbf_phi(x):
    return [gaussian_rbf(u, 10.)(x) for u in np.linspace(-9, 9, 10)]


(rbf_line, rbf_rmse, rbf_reg) = (
    evaluate_basis(rbf_phi, r"RBF")
)

py.iplot([train_points, rbf_line, poly_line, sin_line, basic_line],
         filename="L19_b_p21")

Is this a better fit?

In [50]:
py.iplot([go.Bar(
            x=[r'$\theta $', r'$\theta x$', 
               r'$\theta_0 x + \theta_1 \sin(x)$', 
               r"Polynomial",
               r"RBF"],
            y=[const_rmse, line_rmse, sin_rmse, poly_rmse, rbf_rmse]
    )], filename="L19_b_p23")

### Refusal to Accept Outliers

In [51]:
def crazy_rbf_phi(x):
    return (
        [gaussian_rbf(u,1.)(x) for u in np.linspace(-9, 9, 30)]
    )

(crazy_rbf_line, crazy_rbf_rmse, crazy_rbf_reg) = (
    evaluate_basis(crazy_rbf_phi, "RBF + Crazy")
)

py.iplot([train_points, crazy_rbf_line, poly_line, sin_line, basic_line], 
         filename="L19_b_p24")

In [52]:
train_bars = go.Bar(name = "Train",
    x=[r'$\theta $', r'$\theta x$', r'$\theta_0 x + \theta_1 \sin(x)$', 
       "Polynomial",
       "RBF",
       "RBF + Crazy"],
    y=[const_rmse, line_rmse, sin_rmse, poly_rmse, rbf_rmse, crazy_rbf_rmse])

py.iplot([train_bars], filename="L19_b_p25")


# Which is the best model?

We started with the objective of minimizing the training loss (error).  As we increased the model sophistication by adding features we were able to fit increasingly complex functions to the data and reduce the loss.  However, is our ultimate goal to minimize training error?  

Ideally we would like to minimize the error we make when making new predictions at unseen values of $X$.  One way to evaluate that error is use a **test dataset** which is distinct from the dataset used to train the model.  Fortunately, we have such a test dataset.


In [53]:
test_data = pd.read_csv("toy_test_data.csv")

test_points = go.Scatter(name = "Test Data", x = test_data['X'], y = test_data['Y'], 
                       mode = 'markers', marker=dict(symbol="cross", color="red"))
py.iplot([train_points, test_points], filename="L19_b_p26")


In [54]:
def test_rmse(phi, reg):
    yhat = reg.predict(np.array([phi(x) for x in test_data['X']]))
    return np.sqrt(sq_loss(test_data['Y'], yhat))


In [55]:
test_bars = go.Bar(name = "Test",
    x=[r'$\theta $', r'$\theta x$', r'$\theta_0 x + \theta_1 \sin(x)$', 
       "Polynomial",
       "RBF",
       "RBF + Crazy"],
    y=[np.sqrt(sq_loss(test_data['Y'], test_data['Y'].mean())), 
       test_rmse(lambda x: [x], line_reg),
       test_rmse(sin_phi, sin_reg),
       test_rmse(poly_phi(15), poly_reg),
       test_rmse(rbf_phi, rbf_reg),
       test_rmse(crazy_rbf_phi, crazy_rbf_reg)]
    )

py.iplot([train_bars, test_bars], filename="L19_b_p27")


**What happened here?**
<br/><br/><br/><br/>

---

This is a very common occurrence in machine learning.  As we increased the model complexity 

# What's happening: _Over-fitting_

As we increase the expressiveness of our model we begin to **over-fit** to the variability in our training data.  That is we are learning patterns that do not **generalize** beyond our training dataset

**Over-fitting** is a key challenge in machine learning and statistical inference.  At it's core is a fundamental trade-off between **bias** and **variance**: _the desire to explain the training data and yet be robust to variation in the training data_.

We will study the **bias-variance** trade-off more in the next lecture but for now we will focus on the trade-off between under fitting and over fitting:

<img src="under_over_fitting.png" width="500px">

---

<br/><br/><br/><br/><br/><br/>

# Train, Test, and Validation Splitting

To manage over-fitting it is essential to split your initial training data into a training and testing dataset.  

<img src="train_test_split.png" width="500px">

# Cross Validation

Before running cross validation split the data into train and test subsets (typically a 90-10 split). **Do not look at the test data until after selecting your final model.**

With the remaining training data:
1. Split the remaining training dataset k-different ways (as in the figure above)
1. For each split train the model on the training fraction and then compute the error (RMSE) on the validation fraction.
1. Average the error across each validation fraction to _estimate_ the test error.

**Questions:**
1. What is this accomplishing?
1. What are the implication on the choice of $k$? 

---

<br/><br/><br/><br/><br/><br/>





# Next time:

We will dig more into the bias variance trade-off and introduce the concept of regularization to parametrically explore the space of model complexity. 